# Modelo Actualizado

cpd00002 -> ATP

cpd00008 -> ADP

cpd00163 -> UDP-xylose

(sin cpd0008 y cpd00163), que son los que hacen el modelo ya no optimice.

In [3]:
import libsbml
import pandas as pd
from cobra.io import read_sbml_model

In [5]:
# Cargar el modelo original
modelo_ori_path = '../Models/PstKB.ori.xml'
# Carga el modelo usado Cobra
modelo_ori = read_sbml_model(modelo_ori_path)
# Crear un lector de SBML
lector = libsbml.SBMLReader()
# Leer el archivo SBML y cargarlo en un objeto SBMLDocument
documento_sbml = lector.readSBML(modelo_ori_path)
# Obtener el modelo el SBML
modelo_ori_sbml = documento_sbml.getModel()
# Leer los datos del Excel
excel_path = '../Data/modelData.xls'
comparative_ofs_sheet = 'Comparative OFs'
# Solo nos interesan las últimas dos columnas ('ID' y 'PstKB_cobra'), ajusta si es necesario
comparative_ofs = pd.read_excel(excel_path, sheet_name=comparative_ofs_sheet, usecols=["ID", "PstKB_cobra"])
# Quitar [c0] o [d0] de los IDs si es necesario
comparative_ofs['ID'] = comparative_ofs['ID'].str.replace(r'\[c0\]|\[d0\]', '', regex=True)
# Convertir el DataFrame a un diccionario para fácil acceso
diccionario_coeficientes = pd.Series(comparative_ofs.PstKB_cobra.values, index=comparative_ofs.ID).to_dict()
# Nombre de las reacciones del modelo
columnas_reacciones = [reaccion.getId() for reaccion in documento_sbml.getModel().getListOfReactions()]
resultados_data = []

In [10]:
def modificar_modelo(documento_sbml, metabolito_id, nuevo_coeficiente):
    modelo_modificado_sbml = documento_sbml.getModel()
    reaccion = modelo_modificado_sbml.getReaction('R_bio1_biomass')
    if reaccion:
        for reactante in reaccion.getListOfReactants():
            especie = modelo_modificado_sbml.getSpecies(reactante.getSpecies())
            if metabolito_id in especie.getId():
                reactante.setStoichiometry(nuevo_coeficiente)
        for producto in reaccion.getListOfProducts():
            especie = modelo_modificado_sbml.getSpecies(producto.getSpecies())
            if metabolito_id in especie.getId():
                producto.setStoichiometry(nuevo_coeficiente)
    return documento_sbml

In [25]:
ids_excluir = ['cpd00002','cpd00008', 'cpd00163']
documento_modificado = documento_sbml.clone()
for metabolito_id, nuevo_coeficiente in diccionario_coeficientes.items():
    if metabolito_id not in ids_excluir:
        # Actualiza el documento_modificado en cada iteración para que los cambios sean acumulativos
        documento_modificado = modificar_modelo(documento_modificado, metabolito_id, nuevo_coeficiente)

libsbml.writeSBMLToFile(documento_modificado, "../Models/PstKB-new2.xml")
modelo_new = read_sbml_model("../Models/PstKB-new2.xml")

# Optimización y comparación de resultados
resultado_original = modelo_ori.optimize().fluxes
resultado_modificado = modelo_new.optimize().fluxes

# Crear un DataFrame para comparar resultados
df_resultados = pd.DataFrame({'Original': resultado_original, 'Modificado': resultado_modificado})

# Guardar los resultados en un archivo CSV
df_resultados.to_csv("../Data/Comparacion_Ori_vs_New2.csv")
df_resultados

,Original,Modificado
rxn00001_c0,-191.417904,-118.822709
rxn00001_d0,0.000000,0.000000
rxn00001_g0,0.000000,0.000000
rxn00001_v0,0.000000,0.000000
rxn00001_w0,0.000000,0.000000
...,...,...
EX_cpd00099_e0,-17.698141,-17.563345
EX_cpd00009_e0,-29.631576,-49.593370
EX_cpd11632_e0,-1000.000000,-1000.000000
DM_cpd02701_c0,0.008428,0.008363


In [23]:
biomass_ori = modelo_ori.optimize().objective_value
biomass_new = modelo_new.optimize().objective_value
compareBIOMASS = pd.DataFrame({'Original': [biomass_ori], 'Modificado': [biomass_new]},index=['Biomasa'])
compareBIOMASS

,Original,Modificado
Biomasa,84.276862,83.634975
